# Experiments to test the relationship between various parameters

In [ ]:
import json
from pipeline import get_results
from helper import read_test_results, display_dicts_as_table

with open('config/system_params.json') as f:
  system_params = json.load(f)

# 1 Baseline static and dynamic

In [ ]:
with open('config/baseline_static.json') as f:
    baseline_hyperparameters = json.load(f)
get_results(baseline_hyperparameters, system_params)

In [ ]:
with open('config/baseline_dynamic.json') as f:
    hyperparameters = json.load(f)
get_results(hyperparameters, system_params)

In [ ]:
static_results = read_test_results('baseline_static')
dynamic_results = read_test_results('baseline_dynamic')

display_dicts_as_table([static_results, dynamic_results], ["Static", "Dynamic"])

## Insights
- Accuracy is about the same
- Dynamic graphs take a lot longer time to train
- Dynamic graphs have more significant memory savings

# 2 MLP vs LSTM

In [ ]:
with open('config/lstm_static.json') as f:
    hyperparameters = json.load(f)

get_results(hyperparameters, system_params)

In [ ]:
with open('config/lstm_dynamic.json') as f:
    hyperparameters = json.load(f)

get_results(hyperparameters, system_params)

In [12]:
static_results = read_test_results('baseline_static')
dynamic_results = read_test_results('baseline_dynamic')

static_lstm_results = read_test_results('lstm_static')
dynamic_lstm_results = read_test_results('lstm_dynamic')

display_dicts_as_table([static_results, static_lstm_results, dynamic_results, dynamic_lstm_results], ["Static", "Static LSTM", "Dynamic", "Dynamic LSTM"])

+--------------+--------------------+--------------------+------------------+--------------+-----------------------+------------------+------------------+--------------------------+---------------+----------------+--------------+---------------+----------------+-----------------+
| Name         |   F1 Score (Macro) |   F1 Score (Micro) |   Final accuracy |   Final loss |   Final tensor memory |   Inference MACs |   Memory savings |   Original tensor memory | Saved Nodes   | Saved Spikes   |   Time taken | Total Nodes   | Total Spikes   |   Training MACs |
+==============+====================+====================+==================+==============+=======================+==================+==================+==========================+===============+================+==============+===============+================+=================+
| Static       |           0.706512 |           0.709275 |          70.9275 |    0.0587753 |               8.57086 |      3.70541e+09 |          5.14252 |   

## Insights
- LSTM has higher accuracy than MLP
- Takes about 20x the amount of time
- Best results is static LSTM

# 3 Impact of the number of time steps on static graphs

In [ ]:
timesteps = [1, 2, 5, 10, 30]

for t in timesteps:
    with open(f'config/static_ts{t}.json') as f:
        hyperparameters = json.load(f)
    get_results(hyperparameters, system_params)

In [ ]:
ts1_results = read_test_results('static_ts1')
ts2_results = read_test_results('static_ts2')
ts5_results = read_test_results('static_ts5')
ts10_results = read_test_results('static_ts10')
ts20_results = read_test_results('baseline_static')
ts30_results = read_test_results('static_ts30')

display_dicts_as_table([ts1_results, ts2_results, ts5_results, ts10_results, ts20_results, ts30_results], ["ts=1", "ts=2", "ts=5", "ts=10", "ts=20", "ts=30"])

## Insights
- Loss decreases with the number of time steps and is at a minima somewhere around 20
- Accuacy (for this dataset) seems to be okay even with lesser time steps
- Memory savings decrease with larger time steps (due to more spikes generated)

# 4 Impact of time steps on dynamic graphs

In [ ]:
time_steps = [1, 2, 3, 4, 5, 10]

for t in time_steps:
    with open(f'config/lstm_dynamic.json') as f:
      hyperparameters = json.load(f)
    hyperparameters['time_steps'] = t
    hyperparameters["hyperparameters_id"] = f"dynamic_ts{t}"
    get_results(hyperparameters, system_params)

In [ ]:
ts1_results = read_test_results('dynamic_ts1')
ts2_results = read_test_results('dynamic_ts2')
ts3_results = read_test_results('dynamic_ts3')
ts4_results = read_test_results('dynamic_ts4')
ts5_results = read_test_results('dynamic_ts5')
ts10_results = read_test_results('lstm_dynamic')

display_dicts_as_table([ts1_results, ts2_results, ts3_results, ts4_results, ts5_results, ts10_results], ["ts=1", "ts=2", "ts=3", "ts=4", "ts=5", "ts=10"])

# 5 Attempting to get the best version of the model
## 5.1 Static
- LSTM
- Time steps: 10

In [ ]:
# Run the best model 5 times and get an average
with open('config/lstm_static.json') as f:
    hyperparameters = json.load(f)
    
    hyperparameters["time_steps"] = 10
    system_params_copy = system_params.copy()
    system_params_copy["num_epochs"] = 50

for i in range(5):
    hyperparameters["hyperparameters_id"] = f"lstm_static_best_run_{i+1}"
    get_results(hyperparameters, system_params_copy)

In [ ]:
# Read the results and average them
for i in range(5):
    results = read_test_results(f'lstm_static_best_run_{i+1}')
    if i == 0:
        average_results = results
    else:
        for key in results:
            average_results[key] += results[key]

for key in average_results:
    average_results[key] /= 5

display_dicts_as_table([average_results], ["Average of 5 runs"])

## 5.2 Dynamic

In [ ]:
# Run the best model 5 times and get an average
with open('config/lstm_dynamic.json') as f:
    hyperparameters = json.load(f)
    
    hyperparameters["time_steps"] = 5
    system_params_copy = system_params.copy()
    system_params_copy["num_epochs"] = 40

for i in range(3):
    hyperparameters["hyperparameters_id"] = f"lstm_dynamic_best_run_{i+1}"
    get_results(hyperparameters, system_params_copy)

In [ ]:
# Read the results and average them
for i in range(3):
    results = read_test_results(f'lstm_dynamic_best_run_{i+1}')
    if i == 0:
        average_results = results
    else:
        for key in results:
            average_results[key] += results[key]  

for key in average_results:
    average_results[key] /= 3

display_dicts_as_table([average_results], ["Average of 3 runs"])

# 6 Reset vs no reset

In [ ]:
with open('config/baseline_dynamic_noreset.json') as f:
    hyperparameters = json.load(f)

get_results(hyperparameters, system_params)

with open('config/baseline_dynamic.json') as f:
    hyperparameters = json.load(f)

get_results(hyperparameters, system_params)

noreset_results = read_test_results('baseline_dynamic_noreset')
reset_results = read_test_results('baseline_dynamic')

display_dicts_as_table([noreset_results, reset_results], ["No Reset", "Reset"])


In [ ]:
with open('config/baseline_dynamic_noreset.json') as f:
    hyperparameters = json.load(f)
    hyperparameters["model"] = "LSTM"
    hyperparameters["time_steps"] = 5
    hyperparameters["hyperparameters_id"] = "baseline_dynamic_noreset_lstm"

get_results(hyperparameters, system_params)

noreset_results = read_test_results('baseline_dynamic_noreset_lstm')
reset_results = read_test_results('lstm_dynamic')

display_dicts_as_table([noreset_results, reset_results, read_test_results('baseline_dynamic_noreset_lstm'), read_test_results('lstm_dynamic')], ["No Reset", "Reset", "No Reset LSTM", "LSTM"])


# Benchmarking

- Dynamic LSTM, Average of 3, using test_size 0.2, 0.4, 0.6

In [11]:
# Benchmarking
# Dynamic LSTM, Average of 3, using test_size 0.2, 0.4, 0.6

with open('config/lstm_dynamic.json') as f:
    hyperparameters = json.load(f)
    hyperparameters["dynamic_reset"] = False
    system_params_copy = system_params.copy()

for test_size in [0.2, 0.4, 0.6]:
    system_params_copy["test_size"] = test_size
    for i in range(3):
        hyperparameters["hyperparameters_id"] = f"lstm_dynamic_test_size_{test_size}_run_{i+1}"
        get_results(hyperparameters, system_params_copy)

/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:277: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


Epoch [1/20], Loss: 1.0905
Epoch [2/20], Loss: 0.8869
Epoch [3/20], Loss: 0.8139
Epoch [4/20], Loss: 0.7453
Epoch [5/20], Loss: 0.6922
Epoch [6/20], Loss: 0.6475
Epoch [7/20], Loss: 0.6004
Epoch [8/20], Loss: 0.5529
Epoch [9/20], Loss: 0.5150
Epoch [10/20], Loss: 0.4655
Epoch [11/20], Loss: 0.4228
Epoch [12/20], Loss: 0.3824
Epoch [13/20], Loss: 0.3441
Epoch [14/20], Loss: 0.3019
Epoch [15/20], Loss: 0.2782
Epoch [16/20], Loss: 0.2362
Epoch [17/20], Loss: 0.2049
Epoch [18/20], Loss: 0.1860
Epoch [19/20], Loss: 0.1856
Epoch [20/20], Loss: 0.1451


/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:277: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


Epoch [1/20], Loss: 1.1010
Epoch [2/20], Loss: 0.8852
Epoch [3/20], Loss: 0.8067
Epoch [4/20], Loss: 0.7442
Epoch [5/20], Loss: 0.6889
Epoch [6/20], Loss: 0.6428
Epoch [7/20], Loss: 0.5954
Epoch [8/20], Loss: 0.5482
Epoch [9/20], Loss: 0.4986
Epoch [10/20], Loss: 0.4628
Epoch [11/20], Loss: 0.4092
Epoch [12/20], Loss: 0.3716
Epoch [13/20], Loss: 0.3287
Epoch [14/20], Loss: 0.2862
Epoch [15/20], Loss: 0.2590
Epoch [16/20], Loss: 0.2257
Epoch [17/20], Loss: 0.2039
Epoch [18/20], Loss: 0.1734
Epoch [19/20], Loss: 0.1578
Epoch [20/20], Loss: 0.1301


/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:277: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


...

Epoch [1/20], Loss: 1.2086
Epoch [2/20], Loss: 0.9474
Epoch [3/20], Loss: 0.8700
Epoch [4/20], Loss: 0.8070
Epoch [5/20], Loss: 0.7521
Epoch [6/20], Loss: 0.7037
Epoch [7/20], Loss: 0.6501
Epoch [8/20], Loss: 0.6121
Epoch [9/20], Loss: 0.5582
Epoch [10/20], Loss: 0.5151
Epoch [11/20], Loss: 0.4793
Epoch [12/20], Loss: 0.4210
Epoch [13/20], Loss: 0.3857
Epoch [14/20], Loss: 0.3450
Epoch [15/20], Loss: 0.3070
Epoch [16/20], Loss: 0.2709
Epoch [17/20], Loss: 0.2539
Epoch [18/20], Loss: 0.2161
Epoch [19/20], Loss: 0.1925
Epoch [20/20], Loss: 0.1771


/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:277: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


Epoch [1/20], Loss: 1.2170
Epoch [2/20], Loss: 0.9391
Epoch [3/20], Loss: 0.8660
Epoch [4/20], Loss: 0.8091
Epoch [5/20], Loss: 0.7461
Epoch [6/20], Loss: 0.6947
Epoch [7/20], Loss: 0.6479
Epoch [8/20], Loss: 0.6050
Epoch [9/20], Loss: 0.5500
Epoch [10/20], Loss: 0.5111
Epoch [11/20], Loss: 0.4744
Epoch [12/20], Loss: 0.4209
Epoch [13/20], Loss: 0.3867
Epoch [14/20], Loss: 0.3364
Epoch [15/20], Loss: 0.3092
Epoch [16/20], Loss: 0.2715
Epoch [17/20], Loss: 0.2364
Epoch [18/20], Loss: 0.2111
Epoch [19/20], Loss: 0.1845
Epoch [20/20], Loss: 0.1577


/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:277: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


Epoch [1/20], Loss: 1.2058
Epoch [2/20], Loss: 0.9333
Epoch [3/20], Loss: 0.8624
Epoch [4/20], Loss: 0.7987
Epoch [5/20], Loss: 0.7486
Epoch [6/20], Loss: 0.6945
Epoch [7/20], Loss: 0.6363
Epoch [8/20], Loss: 0.5906
Epoch [9/20], Loss: 0.5514
Epoch [10/20], Loss: 0.5083
Epoch [11/20], Loss: 0.4460
Epoch [12/20], Loss: 0.4058
Epoch [13/20], Loss: 0.3769
Epoch [14/20], Loss: 0.3359
Epoch [15/20], Loss: 0.2996
Epoch [16/20], Loss: 0.2574
Epoch [17/20], Loss: 0.2328
Epoch [18/20], Loss: 0.2054
Epoch [19/20], Loss: 0.1957
Epoch [20/20], Loss: 0.1643


In [13]:
for test_size in [0.2, 0.4, 0.6]:
    for i in range(3):
        results = read_test_results(f'lstm_dynamic_test_size_{test_size}_run_{i+1}')
        if i == 0:
            average_results = results
        else:
            for key in results:
                average_results[key] += results[key]

    for key in average_results:
        average_results[key] /= 3

    display_dicts_as_table([average_results], [f"Test Size {test_size}"])

+---------------+--------------------+--------------------+------------------+--------------+-----------------------+------------------+------------------+--------------------------+---------------+----------------+--------------+---------------+----------------+-----------------+
| Name          |   F1 Score (Macro) |   F1 Score (Micro) |   Final accuracy |   Final loss |   Final tensor memory |   Inference MACs |   Memory savings |   Original tensor memory |   Saved Nodes |   Saved Spikes |   Time taken |   Total Nodes |   Total Spikes |   Training MACs |
+===============+====================+====================+==================+==============+=======================+==================+==================+==========================+===============+================+==============+===============+================+=================+
| Test Size 0.2 |            0.74831 |           0.750223 |          75.0223 |     0.139353 |               57.8533 |       7.0384e+11 |          312.408 

In [15]:
# Benchmarking
# Dynamic LSTM, Average of 3, using test_size 0.2, 0.4, 0.6

with open('config/baseline_dynamic_noreset.json') as f:
    hyperparameters = json.load(f)
    system_params_copy = system_params.copy()

for test_size in [0.2, 0.4, 0.6]:
    system_params_copy["test_size"] = test_size
    for i in range(3):
        hyperparameters["hyperparameters_id"] = f"mlp_dynamic_test_size_{test_size}_run_{i+1}"
        get_results(hyperparameters, system_params_copy)

/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:277: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


Epoch [1/20], Loss: 1.0774
Epoch [2/20], Loss: 0.8134
Epoch [3/20], Loss: 0.6744
Epoch [4/20], Loss: 0.5504
Epoch [5/20], Loss: 0.4480
Epoch [6/20], Loss: 0.3503
Epoch [7/20], Loss: 0.2708
Epoch [8/20], Loss: 0.2133
Epoch [9/20], Loss: 0.1637
Epoch [10/20], Loss: 0.1460
Epoch [11/20], Loss: 0.1126
Epoch [12/20], Loss: 0.1051
Epoch [13/20], Loss: 0.0965
Epoch [14/20], Loss: 0.1176
Epoch [15/20], Loss: 0.0731
Epoch [16/20], Loss: 0.0782
Epoch [17/20], Loss: 0.0577
Epoch [18/20], Loss: 0.0654
Epoch [19/20], Loss: 0.0741
Epoch [20/20], Loss: 0.0680


/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:277: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


Epoch [1/20], Loss: 1.0758
Epoch [2/20], Loss: 0.8013
Epoch [3/20], Loss: 0.6624
Epoch [4/20], Loss: 0.5407
Epoch [5/20], Loss: 0.4223
Epoch [6/20], Loss: 0.3421
Epoch [7/20], Loss: 0.2743
Epoch [8/20], Loss: 0.2224
Epoch [9/20], Loss: 0.1821
Epoch [10/20], Loss: 0.1286
Epoch [11/20], Loss: 0.1184
Epoch [12/20], Loss: 0.1047
Epoch [13/20], Loss: 0.0947
Epoch [14/20], Loss: 0.0888
Epoch [15/20], Loss: 0.0714
Epoch [16/20], Loss: 0.1118
Epoch [17/20], Loss: 0.0586
Epoch [18/20], Loss: 0.0612
Epoch [19/20], Loss: 0.0863
Epoch [20/20], Loss: 0.0722


/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:277: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


...

Epoch [1/20], Loss: 1.1771
Epoch [2/20], Loss: 0.8133
Epoch [3/20], Loss: 0.6349
Epoch [4/20], Loss: 0.4695
Epoch [5/20], Loss: 0.3501
Epoch [6/20], Loss: 0.2787
Epoch [7/20], Loss: 0.1916
Epoch [8/20], Loss: 0.1483
Epoch [9/20], Loss: 0.1140
Epoch [10/20], Loss: 0.0935
Epoch [11/20], Loss: 0.0886
Epoch [12/20], Loss: 0.0679
Epoch [13/20], Loss: 0.0834
Epoch [14/20], Loss: 0.0646
Epoch [15/20], Loss: 0.0520
Epoch [16/20], Loss: 0.0491
Epoch [17/20], Loss: 0.0632
Epoch [18/20], Loss: 0.0588
Epoch [19/20], Loss: 0.0707
Epoch [20/20], Loss: 0.0400


/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:277: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


Epoch [1/20], Loss: 1.2300
Epoch [2/20], Loss: 0.8349
Epoch [3/20], Loss: 0.6543
Epoch [4/20], Loss: 0.5064
Epoch [5/20], Loss: 0.3813
Epoch [6/20], Loss: 0.2756
Epoch [7/20], Loss: 0.2137
Epoch [8/20], Loss: 0.1449
Epoch [9/20], Loss: 0.1129
Epoch [10/20], Loss: 0.0895
Epoch [11/20], Loss: 0.0819
Epoch [12/20], Loss: 0.0814
Epoch [13/20], Loss: 0.0786
Epoch [14/20], Loss: 0.0552
Epoch [15/20], Loss: 0.0392
Epoch [16/20], Loss: 0.0573
Epoch [17/20], Loss: 0.0725
Epoch [18/20], Loss: 0.0491
Epoch [19/20], Loss: 0.0645
Epoch [20/20], Loss: 0.0697


/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/Users/zhekailow/Documents/University/FYP/submission/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:277: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


Epoch [1/20], Loss: 1.1665
Epoch [2/20], Loss: 0.8124
Epoch [3/20], Loss: 0.6363
Epoch [4/20], Loss: 0.4874
Epoch [5/20], Loss: 0.3518
Epoch [6/20], Loss: 0.2524
Epoch [7/20], Loss: 0.1875
Epoch [8/20], Loss: 0.1421
Epoch [9/20], Loss: 0.1099
Epoch [10/20], Loss: 0.0955
Epoch [11/20], Loss: 0.0832
Epoch [12/20], Loss: 0.0578
Epoch [13/20], Loss: 0.0754
Epoch [14/20], Loss: 0.0813
Epoch [15/20], Loss: 0.0485
Epoch [16/20], Loss: 0.0385
Epoch [17/20], Loss: 0.0427
Epoch [18/20], Loss: 0.0553
Epoch [19/20], Loss: 0.0678
Epoch [20/20], Loss: 0.0829
